In [1]:
from collections import defaultdict

In [2]:
from jsp_benchmarks import JSPInstanceLoader
from jsp_exp import Operation, Job, JSP
import jsp_shaving1990

In [3]:
import main as jsp_shaving1996

In [4]:
def init_heads_and_tails(jsp):
    # NOTE: 正しく動いている
    # initしている時点でnaiveなshavingはしていることになる
    for job in jsp:
        for i in range(len(job)):
            previous_ops, next_ops = job[:i], job[i + 1:]
            job[i].r = sum(op.p for op in previous_ops)
            job[i].q = sum(op.p for op in next_ops)

def collect_ops_processed_by_same_machine(jsp: JSP):
    # NOTE: 正しく動いている
    collected = defaultdict(Job)
    for job in jsp:
        for op in job:
            collected[op.processed_by].append(op)
    return dict(collected)

In [5]:
def update_machine(ops_on_same_machine: Job, T: int, shaving_algo, printdebug=False) -> bool:
    # NOTE: これが最も重要！！！！！正確に動いていることを確認しよう！！！！！
    # NOTE: headとtailの更新は直列に行って良いのか？それとも，並列のほうが良いのか？
    # 普通に考えると直列でも良さそうなので，直列で
    if printdebug or True:
        machine = ops_on_same_machine[0].processed_by
        print(f"machine name is '{machine}'")
        if machine == 3:
            print(f"{'='*50}")
            print('shaving前：',ops_on_same_machine.attrtolist('r'), ops_on_same_machine.attrtolist('p'), ops_on_same_machine.attrtolist('q'))
        # print(f"machine name is '{machine}'")
        # print(ops_on_same_machine)
    r_list, p_list, q_list = [ops_on_same_machine.attrtolist(name) for name in ('r', 'p', 'q')]

    pre_heads, pre_tails = r_list, q_list

    next_heads = shaving_algo.adjust_heads(r_list=pre_heads, p_list=p_list, q_list=q_list, UB=T)
    if not next_heads or any(r > T - p - q for r, p, q in zip(next_heads, p_list, pre_tails)):
        print("ここが原因だよ1!", next_heads)
        print(pre_heads, p_list, pre_tails)
        return None

    if printdebug:
        if any(ph != nh for ph, nh in zip(pre_heads, next_heads)):
            print(f"head shaving executed at machine {ops_on_same_machine[0].processed_by}")
            print(f"pre heads: {pre_heads}")
            print(f"next heads: {next_heads}")

    next_tails = shaving_algo.adjust_tails(r_list=next_heads, p_list=p_list, q_list=pre_tails, UB=T)
    if not next_tails or any(r > T - p - q for r, p, q in zip(next_heads, p_list, next_tails)):
        print("ここが原因だよ2!", next_tails)
        print(next_heads, p_list, pre_tails)
        return None

    if printdebug:
        if any(pt != nt for pt, nt in zip(pre_tails, next_tails)):
            print(f"tail shaving executed at machine {ops_on_same_machine[0].processed_by}")
            print(f"pre tails: {pre_tails}")
            print(f"next tails: {next_tails}")

    ops_on_same_machine.setattr(next_heads, attr_name='r')
    ops_on_same_machine.setattr(next_tails, attr_name='q')

    is_updated = (pre_heads != next_heads) or (pre_tails != next_tails)

    if printdebug:
        print(next_heads, p_list, next_tails)
    if printdebug or True:
        machine = ops_on_same_machine[0].processed_by
        if machine == 3:
            print('shaving後：', next_heads, p_list, next_tails)
            print(f"{'='*50}")
    return is_updated


def propagate_windows(jsp: JSP, printdebug=False) -> None:
    # NOTE: 正しく動いている
    if printdebug:
        print('start propagation')
    for j in range(len(jsp)):
        job = jsp[j]
        if printdebug:
            print(f"job name is '{job.name}'")
        for i in range(len(job) - 1):
            if printdebug:
                print(f"Operation O_{j}{i}, window: [{job[i].r}, {55-job[i].p-job[i].q}]", job[i])
            job[i + 1].r = max(job[i + 1].r, job[i].r + job[i].p)
            job[i].q = max(job[i].q, job[i + 1].q + job[i + 1].p)
    if printdebug:
        print('end propagation')


def icp_shaving(jsp: JSP, T: int, shaving_algo, printdebug=False) -> bool:
    collected = collect_ops_processed_by_same_machine(jsp)
    updated = True
    while updated:
        updated = False
        for ops_on_same_machine in collected.values():
            res = update_machine(ops_on_same_machine, T, shaving_algo, printdebug)
            if res is None:
                return False
            updated = res or updated
        if updated:
            propagate_windows(jsp, printdebug)
    return True

In [19]:
loader = JSPInstanceLoader()
ft06 = loader.load('ft06')
T = ft06.meta_data.optimum

init_heads_and_tails(ft06.jsp)
j0, j1, j2, j3, j4, j5 = ft06.jsp

icp_shaving(ft06.jsp, T-1, jsp_shaving1996, False)

machine name is '2'
machine name is '0'
machine name is '1'
machine name is '3'
shaving前： [10, 43, 5, 15, 24, 3] [7, 4, 4, 3, 1, 3] [9, 0, 25, 17, 0, 24]
shaving後： [10, 43, 5, 15, 24, 3] [7, 4, 4, 3, 1, 3] [9, 0, 25, 17, 0, 24]
machine name is '5'
machine name is '4'
machine name is '2'
machine name is '0'
machine name is '1'
machine name is '3'
shaving前： [14, 44, 5, 15, 28, 3] [7, 4, 4, 3, 1, 3] [11, 0, 32, 17, 0, 33]
shaving後： [14, 44, 5, 15, 28, 3] [7, 4, 4, 3, 1, 3] [11, 0, 32, 17, 0, 33]
machine name is '5'
machine name is '4'
machine name is '2'
machine name is '0'
machine name is '1'
machine name is '3'
shaving前： [19, 44, 5, 18, 28, 3] [7, 4, 4, 3, 1, 3] [11, 0, 32, 17, 0, 33]
shaving後： [19, 44, 5, 18, 28, 3] [7, 4, 4, 3, 1, 3] [11, 0, 32, 17, 0, 33]
machine name is '5'
machine name is '4'
machine name is '2'
machine name is '0'
machine name is '1'
machine name is '3'
shaving前： [22, 44, 5, 18, 35, 3] [7, 4, 4, 3, 1, 3] [11, 0, 34, 17, 0, 35]
shaving後： [22, 44, 5, 18, 35, 3] [7, 

False

In [20]:
l = 0
for job in ft06.jsp:
    for op in job:
        l += T - op.p - op.q - op.r + 1
else:
    print('決定変数の数:',l)

決定変数の数: 270


In [8]:
def _find_center_integer(u: int, v: int) -> int:
    # 一番最初以外は必ず偶数個の整数が含まれているので，ピッタリ中央の整数はない
    # ので，小数点以下を含む中央の値を計算して，その左側の値を中央の値として取り扱うこととする
    # NOTE: 想定通りに動いている
    import math
    c = math.floor((u + v) / 2)
    return c


def get_next_interval(u: int, v: int):
    # NOTE: 想定通りに動いている
    return u, _find_center_integer(u, v)

In [9]:
get_next_interval(u=1, v=2)

(1, 1)